Clustering Temporal Gene Expression Profiles in Evo-Devo-Data 


In [7]:
%matplotlib inline
%config InlineBackend.figure_format = 'png'#'svg' would be better, but eats memory for these big plots.
from matplotlib import pyplot as plt
import numpy as np
import GPy
import sys
import os
import pandas as pd
sys.path.append('/Users/agkaessmann/Desktop/Gp_clust/')
import GPclust

Loaded backend module://matplotlib_inline.backend_inline version unknown.


In [8]:
nameAnalysis= # Name of analysis
today= # Date
outdir= # Output directory
if not os.path.exists(outdir):
    os.makedirs(outdir)

In [ ]:
# read in as dataframes
expression = pd.read_csv('/Users/agkaessmann/Desktop/Gp_clust/010422_Mouse_liver.txt', delimiter="\t", header=0, index_col=0) # Expression data (in our case fold difference)
times = pd.read_csv('/Users/agkaessmann/Desktop/Gp_clust/Times4Brain.txt', delimiter="\t", header=0, index_col=0) # File with time points

#get gene names
gene_names = expression.index.values
# turn to matrix
times = times.values

expression= expression.values
#control dimensions
print(expression.shape)
print(times.shape)
print(times.T)

First step: Standardize Gene-wise

In [ ]:
expression -= expression.mean(1)[:,np.newaxis]
expression /= expression.std(1)[:,np.newaxis]

In [ ]:
plt.plot(times,expression[4,:] , 'ro')

In [15]:
k_underlying = GPy.kern.Matern52(input_dim=1, variance=1.0, lengthscale=7.)
k1=GPy.kern.Matern52(input_dim=1, variance=0.1, lengthscale=7.)
k2=GPy.kern.White(1, variance=0.3)
#avoid under-estimating (non-structured noise)
k2.variance.fix(0.5)
k_corruption = k1+ k2


In [16]:
m = GPclust.MOHGP(times, k_underlying, k_corruption, expression, K=20, prior_Z='DP', alpha=1e-6)
#m.kernF.lengthscale.fix(6)

In [ ]:
m.hyperparam_opt_interval = 100 # how often to optimize the hyperparameters
m.hyperparam_opt_args['messages'] = True # turn off the printing of the optimization
m.optimize()
m.systematic_splits(verbose=True)

Converged?

In [ ]:
m.reorder()# plot the biggest clusters at the top
plt.figure(figsize=(20,20))
m.plot(on_subplots=True, colour=True, in_a_row=False, newfig=False, min_in_cluster=1, joined=False, ylim=(-4,4), data_in_replicate=False)

plt.savefig(outdir+"/Clusters"+nameAnalysis+'.png')

We can also inspect the optimized values of the kernel parameters

In [ ]:
print (m)

Get posterior probability of assiging nth gene to kth cluster?

Is it possible to specify the number of clusters wanted?

In [ ]:
m.phi.shape
noClusters=m.phi.shape[1]
noGenes=m.phi.shape[0]
print(noClusters)

Posterior Mean

In [ ]:
print(m.muk.shape)
plt.plot(times,m.muk[:,1])
PostMean=pd.DataFrame(m.muk)
PostMean.to_csv(outdir+"/PostMean_"+nameAnalysis+".csv")

Posterior Variance

In [ ]:
m.Cs[1]

Create Dataframe with rows =genes and columns=membership probability and export as csv.

In [ ]:
for i in range(noClusters):
    PostMean=pd.DataFrame(m.Lambda_inv[i])
    PostMean.to_csv(outdir+"/PostCov_"+str(i+1)+nameAnalysis+".csv")

In [ ]:
ClusterProb=pd.DataFrame(m.phi, index= gene_names)
ClusterProb.to_csv(outdir+"/ClusterPobabilities_"+nameAnalysis+".csv")

Have a look at some of the genes

In [ ]:
plt.plot(times,expression[2,:] , 'ro')
np.argmax(m.phi[2,:])

In [ ]:
plt.plot(times,expression[1,:] , 'ro')
np.argmax(m.phi[1,:])

In [ ]:
plt.plot(times,expression[2,:] , 'ro')
np.argmax(m.phi[2,:])

Genes assigned to each cluster

In [ ]:
ClusterAss=[]
for i in range(noClusters):
    print("\n Gene with highest probability to be in Cluster "+str(i)+" are")
    for j in range(noGenes): 
        if np.argmax(m.phi[j,:])==i:
                    print(gene_names[j],j)
        ClusterAss.append(np.argmax(m.phi[j,:]))
        #print(ClusterAss[j],gene_names[j],j)
